In [2]:
# ============================================================
# Notebook setup: run this before everything
# ============================================================
# -- Copied from lecture
%load_ext autoreload
%config IPCompleter.greedy=True
%autoreload 1
%aimport util
import logging

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

from util import util

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 4)

raw_data = util.load_dataset('7_gecco2019_train_water_quality.csv')
val_data = util.load_dataset('8_gecco2019_valid_water_quality.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Gaussian Mixture Models
Gaussian Mixture Models (GMM) are a popular unsupervised learning algorithm that can be used to model the distribution of a dataset. In the context of anomaly detection, GMM can be used to find clusters of normal data points and identify anomalies. GMMs describe the distribution via a weighted sum of Gaussian components.

GMMs assume, that data is generated by the following probabilistic model:
$$
X_Z,
$$
where both $Z$ and $X_Z$ are random variables. $Z$ is a latent variable that represents the component of the data, while $X_Z$ is the observed data. The latent variable $Z$ is assumed to be generated by a probability distribution $p(Z)$, while $X_Z$ follows a multivariate Gaussian distribution.

In mathematical terms, a GMM is a probability distribution that can be represented as:
$$
g(x, \mu, \Sigma, \tau) = \sum_{k=1}^{n} \tau_{k} \mathcal{f}(x, \mu_{k}, \Sigma_{k}),
$$
where $\tau$ is a vector of weights, $\mu$ is a vector of means, $\Sigma$ is a covariance matrix, and $\mathcal{f}$ is the Gaussian probability density function.

## Preprocessing
In order to use GMM for anomaly detection, we first need to make sure, that our data is free from missing values. As seen before, a linear interpolation approach yields the best results. Therefore, we will interpolate missing values using this method. Then, we have to apply a sliding window approach using the `aggregation_length` parameter explained above and aggregate the data into windows. This makes sure that we capture temporal correlations between data points and additionally removes noise in the data. The final step of our preprocessing pipeline is to standardize the data.

In [3]:
# Preprocess the data (interpolating missing values and applying sliding window)
gmm_data_df = util.impute_missing_values(raw_data)
gmm_data_df = util.apply_sliding_window_and_aggregate(gmm_data_df)

# Identify the features to be used for GMM
gmm_features = util.get_feature_columns(gmm_data_df)

# Standardize the data (GMM assumes normally distributed data)
gmm_scaler = StandardScaler()
gmm_data_df[gmm_features] = gmm_scaler.fit_transform(gmm_data_df[gmm_features])

print(gmm_data_df.head())

# Preprocess validation data
gmm_val_data_df = util.impute_missing_values(val_data)
gmm_val_data_df = util.apply_sliding_window_and_aggregate(gmm_val_data_df)

# Standardize the data (KDE assumes normally distributed data)
gmm_val_data_df[gmm_features] = gmm_scaler.transform(gmm_val_data_df[gmm_features])

                     window_0  window_1  window_2  window_3  window_4  \
Time                                                                    
2017-07-01 00:09:00 -1.231517  1.060557 -0.382443 -0.408179 -1.684389   
2017-07-01 00:10:00 -1.242696  1.034397 -0.350244 -0.192011 -1.673169   
2017-07-01 00:11:00 -1.231517  0.982218 -0.342876 -0.315704 -1.696363   
2017-07-01 00:12:00 -1.231517  0.982218 -0.333053 -0.365718 -1.681567   
2017-07-01 00:13:00 -1.231517  1.008378 -0.328141 -0.076374 -1.697380   

                     window_5  window_6  window_7  window_8  window_9  ...  \
Time                                                                   ...   
2017-07-01 00:09:00 -2.181417 -1.242701  1.034405 -0.350237 -0.192011  ...   
2017-07-01 00:10:00 -2.209378 -1.231522  0.982226 -0.342870 -0.315704  ...   
2017-07-01 00:11:00 -2.195122 -1.231522  0.982226 -0.333047 -0.365719  ...   
2017-07-01 00:12:00 -2.215175 -1.231522  1.008386 -0.328135 -0.076374  ...   
2017-07-01 00:13:00 

## Determine the number of Gaussians
Next, we need to determine the number of Gaussians to use for our GMM. We can do this using the Bayesian Information Criterion (BIC) or the elbow method. The BIC is a measure of the model's goodness of fit, while the elbow method is a visual tool that helps us determine the optimal number of Gaussians. In our case, we will use the BIC method.

In [4]:
# Select only relevant features (mean and variance).
X = gmm_data_df[gmm_features]
X_val = gmm_val_data_df[gmm_features]

# Fit GMM and determine optimal K using BIC.
lowest_bic = np.inf
best_k = None

# Try GMMs with 1 to 10 components.
for k in range(1, 11):
    gmm = GaussianMixture(n_components=k, covariance_type='full', random_state=42)
    gmm.fit(X)

    bic = gmm.bic(X_val)

    # Do we have a better model?
    if bic < lowest_bic:
        lowest_bic = bic
        best_k = k

print(f"Best GMM with {best_k} components, BIC: {lowest_bic:.2f}")

Best GMM with 3 components, BIC: 580398095.80


## Train the GMM
Next, we train the GMM with the optimal number of components. Additionally, we compute the log likelihood scores for each data point. This score is a measure of how likely a data point is to be generated by the Gaussian distribution. Higher scores indicate a higher likelihood of being generated by a Gaussian distribution.

In [6]:
# Train GMM with best K.
best_k = 3
final_gmm = GaussianMixture(n_components=best_k, covariance_type='full', random_state=42)
final_gmm.fit(X)

# Compute likelihood scores for the training data.
log_likelihood = final_gmm.score_samples(X)

## Threshold Optimization
Now, we need to define a threshold to separate normal data from anomalous data. We will use a simple threshold optimization approach. First, we define the percentiles to test. Then, we compute precision, recall, and F1-score for each percentile. These are the preferred metrics when working with big class imbalances. Finally, we select the percentile with the highest F1-score.

In [7]:
# Compute likelihood scores for the training data.
log_likelihood_val = final_gmm.score_samples(X_val)

# Define percentiles to test.
percentiles = np.arange(0.1, 2.1, 0.1)

# For storing the results.
results = []

for p in percentiles:
    # Get predictions and threshold
    y_pred, threshold = util.get_predictions_from_log_likelihood(log_likelihood_val, p)

    # Compute performance
    f1, precision, recall = util.compute_model_performance(y_pred, gmm_val_data_df['Event'])

    # Store results.
    results.append((p, threshold, precision, recall, f1))

# Convert to DataFrame for better visualization.
df_results = pd.DataFrame(results, columns=['Percentile', 'Threshold', 'Precision', 'Recall', 'F1-score'])

# Display results.
gmm_best_percentile = df_results.loc[df_results['F1-score'].idxmax()]
print(df_results)
print(f"Best GMM model with percentile {gmm_best_percentile['Percentile']} and threshold {gmm_best_percentile['Threshold']} achieves an F1-score of {gmm_best_percentile['F1-score']}")

    Percentile     Threshold  Precision    Recall  F1-score
0          0.1 -42684.072814   0.000000  0.000000  0.000000
1          0.2 -40559.058023   0.000000  0.000000  0.000000
2          0.3 -38488.776691   0.000000  0.000000  0.000000
3          0.4 -36473.229550   0.000000  0.000000  0.000000
4          0.5 -34512.419886   0.000000  0.000000  0.000000
5          0.6 -32606.343682   0.000000  0.000000  0.000000
6          0.7 -30755.000937   0.000000  0.000000  0.000000
7          0.8 -28958.393113   0.000000  0.000000  0.000000
8          0.9 -27216.522036   0.000000  0.000000  0.000000
9          1.0 -25529.384419   0.000000  0.000000  0.000000
10         1.1 -23896.980261   0.000000  0.000000  0.000000
11         1.2 -22319.311754   0.000000  0.000000  0.000000
12         1.3 -20796.379264   0.000000  0.000000  0.000000
13         1.4 -19328.180234   0.000000  0.000000  0.000000
14         1.5 -17914.714663   0.000000  0.000000  0.000000
15         1.6 -16555.985474   0.000000 

## Compute Number of Anomalies
Now, we compute the number of anomalies for the identified threshold.

In [8]:
# Compute number of anomalies
threshold = np.percentile(log_likelihood, gmm_best_percentile['Percentile'])
anomalies = log_likelihood < threshold

print(f"Anomaly threshold: {threshold:.2f}")
print(f"Number of anomalies: {np.sum(anomalies)}")

# Add anomaly labels to DataFrame
gmm_data_df['Anomaly_Score'] = log_likelihood
gmm_data_df['Anomaly'] = anomalies

print(gmm_data_df['Anomaly_Score'].head())

Anomaly threshold: 99.43
Number of anomalies: 2517
Time
2017-07-01 00:09:00    153.153172
2017-07-01 00:10:00    155.260781
2017-07-01 00:11:00    157.076251
2017-07-01 00:12:00    152.309765
2017-07-01 00:13:00    149.886448
Name: Anomaly_Score, dtype: float64


## Confusion Matrix and Classification Report
To measure the performance of our anomaly detection model on the training data, we can use the confusion matrix and classification report.

In [9]:
# Convert Boolean to integers for evaluation
y_true = gmm_data_df['Event'].astype(int)  # Actual contamination events
y_pred = gmm_data_df['Anomaly'].astype(int)  # Detected anomalies

# Print performance metrics
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred))

Confusion Matrix:
 [[129919   2223]
 [    35    294]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99    132142
           1       0.12      0.89      0.21       329

    accuracy                           0.98    132471
   macro avg       0.56      0.94      0.60    132471
weighted avg       1.00      0.98      0.99    132471



The GMM performed well on the training data with a high overall accuracy of 98%, but this is primarily due to the class imbalance that favors normal samples. The model correctly classified nearly all normal cases (TN = 129,919) with a precision of 1.00. This indicates relatively few false positives (FP = 2,223). However, while it detected 89% of actual anomalies (recall = 0.89), it still missed 11% (FN = 35). The precision for anomalies (0.12) suggests that only a small portion of the detected anomalies were true contaminations, meaning there are still many false alarms. The F1-score of 0.21 shows an imbalance between precision and recall. We can conclude that the model is moderately effective at detecting anomalies in the training set, but further improvements may be needed for generalization to unseen data.

## Performance on Test Data
The lack of generalization can be seen when we test the trained GMM on the test data. First, we have to perform the same preprocessing steps as with the training data. Afterwards, we can apply the GMM to the test data and compute the confusion matrix and classification report.

In [10]:
# Load test data
test_data = util.load_dataset('6_gecco2019_test_water_quality.csv')

# Preprocess test data
gmm_test_data_df = util.impute_missing_values(test_data)
gmm_test_data_df = util.apply_sliding_window_and_aggregate(gmm_test_data_df)

# Standardize the data (GMM assumes normally distributed data)
gmm_test_data_df[gmm_features] = gmm_scaler.transform(gmm_test_data_df[gmm_features])

# Compute scores for test data
X_test = gmm_test_data_df[gmm_features]
log_likelihood_test = final_gmm.score_samples(X_test)  # Higher is more normal, lower is more anomalous

threshold_test = np.percentile(log_likelihood_test, gmm_best_percentile['Percentile'])
anomalies_test = log_likelihood_test < threshold_test

# Add anomaly labels to DataFrame
gmm_test_data_df['Anomaly_Score'] = log_likelihood_test
gmm_test_data_df['Anomaly'] = anomalies_test

# Convert Boolean to integers for evaluation
y_true = gmm_test_data_df['Event'].astype(int)  # Actual contamination events
y_pred = gmm_test_data_df['Anomaly'].astype(int)  # Detected anomalies

# Print performance metrics
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred))

Confusion Matrix:
 [[30788   554]
 [  251    48]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99     31342
           1       0.08      0.16      0.11       299

    accuracy                           0.97     31641
   macro avg       0.54      0.57      0.55     31641
weighted avg       0.98      0.97      0.98     31641



On the test data, the performance of the GMM dropped significantly, with anomaly recall dropping to 16%. The model failed to detect most contaminants (FN = 251). While normal samples were still well classified (98% accuracy for class 0), anomaly precision was only 8%, indicating a high false positive rate. The F1-score of 0.11 confirms that the model struggles to generalize, probably due to overfitting on training data. These results suggest that the model needs better generalization techniques, such as adjusting the threshold, retraining with a more balanced dataset, or considering alternative anomaly detection methods.